Import Python modules.

In [54]:
import json
import pandas as pd
import numpy as np


1. Read detailed postseason game stats JSON file into a viewable pandas object.
2. Split detailed postseason panda object into an [1] "id" and [2] "teams" pandas objects.
3. Split "teams" pandas for each game into four (4) lists...
   * [1] team_0, [2] team_0.location, [3] team_1, and [4] team_1.location

In [78]:
# load detailed weekly_games game stats JSON as a pandas object
cfb_year = input('Input college football year?')
week_number = input('Input regular season week number')

# 2024-regular-weekNumber-detailedGamesCFB-JSON

json_detailed_weekly_games = cfb_year + '_cfb_JSON-CSV/' + cfb_year + '-regular-' + week_number + '-detailedGamesCFB-JSON.json'
detailed_weekly_games = pd.read_json(json_detailed_weekly_games)

# split detailed_postseason pandas object to list id_only (N-games x 1)
id_only = detailed_weekly_games.drop(['teams'], axis=1)
rows = id_only.shape[0]
print(str(rows) + ' games')

# split detailed_postseason pandas object to list teams (N-games x 1)
teams_only = detailed_weekly_games.drop(['id'], axis=1)

# split detailed_postseason 'DataFrame' into a [1] teams[j][0]['teams_key'] list
# & teams[j][1]['teams_key'] list (N-teams x 1)
def teams_column(all_games, team_number, teams_key):
    teams_list = []
    for game in range(all_games):
        teams_list.append((teams_only['teams'][game][team_number][teams_key]))
    return teams_list

team0_list = teams_column(rows, 0, 'school')
team1_list = teams_column(rows, 1, 'school')
team0_location_list = teams_column(rows, 0, 'homeAway')
team1_location_list = teams_column(rows, 1, 'homeAway')
team0_points_list = teams_column(rows, 0, 'points')
team1_points_list = teams_column(rows, 1, 'points')
team0_school_id_list = teams_column(rows, 0, 'schoolId')
team1_school_id_list = teams_column(rows, 1, 'schoolId')

# Combine the 4 list-Series into a DataFrame (N-teams x 4)
teams_detail = pd.DataFrame({'team0': team0_list, 'team0.id': team0_school_id_list, 'team0.location': team0_location_list,
                             'team0.points': team0_points_list, 'team1' : team1_list,  'team1.id': team1_school_id_list,
                             'team1.location' : team1_location_list, 'team1.points': team1_points_list})

# Join games id with team_0 and team_1 information
id_teams_locations_df = id_only.join(teams_detail, on=None)
id_teams_locations_df


Input college football year? 2024
Input regular season week number 4


54 games


,id,team0,team0.id,team0.location,team0.points,team1,team1.id,team1.location,team1.points
0,401628979,Miami (OH),193,away,3,Notre Dame,87,home,28
1,401636876,Utah,254,away,22,Oklahoma State,197,home,19
2,401636872,BYU,252,home,38,Kansas State,2306,away,9
3,401636874,Colorado,38,home,38,Baylor,239,away,31
4,401636875,Arkansas State,2032,away,7,Iowa State,66,home,52
5,401636877,Arizona State,9,away,22,Texas Tech,2641,home,30
6,401636879,West Virginia,277,home,32,Kansas,2305,away,28
7,401635557,SMU,2567,home,66,TCU,2628,away,42
8,401635555,Miami,2390,away,50,South Florida,58,home,15
9,401640991,Liberty,2335,home,35,East Carolina,151,away,24


In [79]:
# practicing reading JSON file with nested dictionaries
# read detailed category stats from game #1, team0 [game0 until last row][team0 or team1] (read values from a "stats" dictionary)
# detailed stats list for each team will vary

# loop through every game as a new row; join rows with game id (id_detailed_game_stats_df);
#   join "id_team_locations_df" with "id_detailed_game_stats_df"

accumulator = []
for game in range(rows):
    for team in range(2):
        team_detailed_stats = teams_only['teams'][game][team]['stats']
        
        # convert team_detailed_stats into a DataFrame
        team_detailed_stats_df = pd.DataFrame(team_detailed_stats)
        
        # convert team_detailed_stats_df into a numpy nested array
        team_detailed_stats_np = team_detailed_stats_df.to_numpy()
        
        # transpose nested array
        team_detailed_stats_np = np.transpose(team_detailed_stats_np)
        
        # create team name (either 'team0.' or 'team1.')
        team_int = 'team' + str(team) + '.'
        
        # convert back to DataFrame with header as stats columns
        team_detailed_stats_df2 = pd.DataFrame(team_detailed_stats_np)
        new_header = team_detailed_stats_df2.iloc[0]
        team_detailed_stats_df2 = team_detailed_stats_df2[1:]
        team_detailed_stats_df2.columns = new_header
        
        # insert team0. or team1. as a prefix in the header row
        team_detailed_stats_df2 = team_detailed_stats_df2.add_prefix(team_int)
        accumulator.append(team_detailed_stats_df2)

# accumulator includes all teams, odd rows are team0, even rows are team1
big_df = pd.concat(accumulator)
big_df


,team0.rushingTDs,team0.puntReturnYards,team0.puntReturnTDs,team0.puntReturns,team0.passingTDs,team0.kickingPoints,team0.fumblesRecovered,team0.totalFumbles,team0.tacklesForLoss,team0.defensiveTDs,...,team1.totalYards,team1.fourthDownEff,team1.thirdDownEff,team1.firstDowns,team0.kickReturnYards,team0.kickReturnTDs,team0.kickReturns,team0.interceptionYards,team0.interceptionTDs,team0.passesIntercepted
1,0,10,0,1,0,3,2,2,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,428,1-2,5-11,24,NaN,NaN,NaN,NaN,NaN,NaN
1,1,11,0,3,1,10,0,2,3,0,...,NaN,NaN,NaN,NaN,23,0,2,0,0,2
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,285,0-0,4-15,11,NaN,NaN,NaN,NaN,NaN,NaN
1,1,90,1,1,2,8,1,0,3,2,...,NaN,NaN,NaN,NaN,18,0,1,15,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,503,0-0,7-9,28,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,2,9,0,2,8,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,442,0-1,7-17,22,NaN,NaN,NaN,NaN,NaN,NaN
1,1,12,0,2,1,11,2,2,11,0,...,NaN,NaN,NaN,NaN,0,0,1,17,0,1


group big_df every two rows

In [80]:
group_big_df = big_df.groupby(np.arange(len(big_df))//2).sum()
group_big_df


,team0.rushingTDs,team0.puntReturnYards,team0.puntReturnTDs,team0.puntReturns,team0.passingTDs,team0.kickingPoints,team0.fumblesRecovered,team0.totalFumbles,team0.tacklesForLoss,team0.defensiveTDs,...,team1.totalYards,team1.fourthDownEff,team1.thirdDownEff,team1.firstDowns,team0.kickReturnYards,team0.kickReturnTDs,team0.kickReturns,team0.interceptionYards,team0.interceptionTDs,team0.passesIntercepted
0,0,10,0,1,0,3,2,2,2,0,...,428,1-2,5-11,24,0,0,0,0,0,0
1,1,11,0,3,1,10,0,2,3,0,...,285,0-0,4-15,11,23,0,2,0,0,2
2,1,90,1,1,2,8,1,0,3,2,...,367,0-3,8-17,17,18,0,1,15,0,2
3,3,7,0,4,2,8,0,1,5,0,...,314,1-2,7-17,15,0,0,0,0,0,0
4,0,0,0,0,1,1,0,3,5,0,...,490,2-2,6-11,25,31,0,2,30,0,2
5,3,0,0,0,0,4,0,0,3,0,...,337,0-0,8-19,24,26,0,2,0,0,0
6,2,18,0,2,2,6,1,1,5,0,...,432,2-2,6-13,25,15,0,1,16,0,1
7,3,69,1,1,2,16,3,1,5,3,...,480,3-4,8-16,24,51,0,3,97,1,2
8,4,0,0,0,3,6,0,0,7,0,...,365,1-1,7-17,22,70,0,3,0,0,2
9,1,0,0,0,4,5,1,3,5,0,...,331,1-2,7-15,20,55,0,3,35,0,1


In [81]:
# view header stats
group_big_df.columns


Index(['team0.rushingTDs', 'team0.puntReturnYards', 'team0.puntReturnTDs',
       'team0.puntReturns', 'team0.passingTDs', 'team0.kickingPoints',
       'team0.fumblesRecovered', 'team0.totalFumbles', 'team0.tacklesForLoss',
       'team0.defensiveTDs', 'team0.tackles', 'team0.sacks', 'team0.qbHurries',
       'team0.passesDeflected', 'team0.possessionTime', 'team0.interceptions',
       'team0.fumblesLost', 'team0.turnovers', 'team0.totalPenaltiesYards',
       'team0.yardsPerRushAttempt', 'team0.rushingAttempts',
       'team0.rushingYards', 'team0.yardsPerPass', 'team0.completionAttempts',
       'team0.netPassingYards', 'team0.totalYards', 'team0.fourthDownEff',
       'team0.thirdDownEff', 'team0.firstDowns', 'team1.rushingTDs',
       'team1.puntReturnYards', 'team1.puntReturnTDs', 'team1.puntReturns',
       'team1.passingTDs', 'team1.kickReturnYards', 'team1.kickReturnTDs',
       'team1.kickReturns', 'team1.kickingPoints', 'team1.interceptionYards',
       'team1.interceptionT

In [82]:
# Concatenate weekly regular season ID DataFrame and detailed stats DataFrame
cfb_weekly_games = pd.concat([id_teams_locations_df, group_big_df], axis=1)
cfb_weekly_games


,id,team0,team0.id,team0.location,team0.points,team1,team1.id,team1.location,team1.points,team0.rushingTDs,...,team1.totalYards,team1.fourthDownEff,team1.thirdDownEff,team1.firstDowns,team0.kickReturnYards,team0.kickReturnTDs,team0.kickReturns,team0.interceptionYards,team0.interceptionTDs,team0.passesIntercepted
0,401628979,Miami (OH),193,away,3,Notre Dame,87,home,28,0,...,428,1-2,5-11,24,0,0,0,0,0,0
1,401636876,Utah,254,away,22,Oklahoma State,197,home,19,1,...,285,0-0,4-15,11,23,0,2,0,0,2
2,401636872,BYU,252,home,38,Kansas State,2306,away,9,1,...,367,0-3,8-17,17,18,0,1,15,0,2
3,401636874,Colorado,38,home,38,Baylor,239,away,31,3,...,314,1-2,7-17,15,0,0,0,0,0,0
4,401636875,Arkansas State,2032,away,7,Iowa State,66,home,52,0,...,490,2-2,6-11,25,31,0,2,30,0,2
5,401636877,Arizona State,9,away,22,Texas Tech,2641,home,30,3,...,337,0-0,8-19,24,26,0,2,0,0,0
6,401636879,West Virginia,277,home,32,Kansas,2305,away,28,2,...,432,2-2,6-13,25,15,0,1,16,0,1
7,401635557,SMU,2567,home,66,TCU,2628,away,42,3,...,480,3-4,8-16,24,51,0,3,97,1,2
8,401635555,Miami,2390,away,50,South Florida,58,home,15,4,...,365,1-1,7-17,22,70,0,3,0,0,2
9,401640991,Liberty,2335,home,35,East Carolina,151,away,24,1,...,331,1-2,7-15,20,55,0,3,35,0,1


In [83]:
# create a DataFrame by each team and display the teams's total offense
# id | team0 | team0.points | team0.totalYards | team0.totalDefenseYards |
# team1 | team1.points | team1.totalYards | team1.totalDefenseYards
cfb_weekly_total_Yards = cfb_weekly_games[['id', 'team0', 'team0.id', 'team0.totalYards', 'team1.totalYards',
                                                  'team1', 'team1.id', 'team1.totalYards', 'team0.totalYards']]
cfb_weekly_total_Yards.columns = ['id', 'team0', 'team0.id', 'team0.totalYards', 'team0.defenseYards',
                                                  'team1', 'team1.id', 'team1.totalYards', 'team1.defenseYards']
cfb_weekly_total_Yards


,id,team0,team0.id,team0.totalYards,team0.defenseYards,team1,team1.id,team1.totalYards,team1.defenseYards
0,401628979,Miami (OH),193,229,428,Notre Dame,87,428,229
1,401636876,Utah,254,456,285,Oklahoma State,197,285,456
2,401636872,BYU,252,241,367,Kansas State,2306,367,241
3,401636874,Colorado,38,432,314,Baylor,239,314,432
4,401636875,Arkansas State,2032,182,490,Iowa State,66,490,182
5,401636877,Arizona State,9,376,337,Texas Tech,2641,337,376
6,401636879,West Virginia,277,444,432,Kansas,2305,432,444
7,401635557,SMU,2567,375,480,TCU,2628,480,375
8,401635555,Miami,2390,592,365,South Florida,58,365,592
9,401640991,Liberty,2335,414,331,East Carolina,151,331,414


In [84]:
# Create a DataFrame sorted by team0.id (N rows x 4 columns)
# Columns: team | team_id | offense_yards | defense_yards
team0_total_yards = cfb_weekly_total_Yards[['team0.id', 'team0', 'team0.totalYards', 'team1.totalYards']]
team0_total_yards.columns = ['id', 'team_wk4', 'Off_wk4', 'Def_wk4']

# Create a DataFrame sorted by team1.id (N rows x 4 columns)
team1_total_yards = cfb_weekly_total_Yards[['team1.id', 'team1', 'team1.totalYards', 'team0.totalYards']]
team1_total_yards.columns = ['id', 'team_wk4', 'Off_wk4', 'Def_wk4']

# Append rows of team0 & team1 DataFrames
actual_off_def = pd.concat([team0_total_yards, team1_total_yards])

# Write DataFrame to a CSV file
actual_off_def.to_csv('2024_cfb_JSON-CSV/2024-week4-teams-off-def.csv', index=False)
actual_off_def


,id,team_wk4,Off_wk4,Def_wk4
0,193,Miami (OH),229,428
1,254,Utah,456,285
2,252,BYU,241,367
3,38,Colorado,432,314
4,2032,Arkansas State,182,490
...,...,...,...,...
49,167,New Mexico,485,345
50,2,Auburn,431,334
51,57,Florida,503,480
52,142,Missouri,442,324


In [85]:
# find certain team(s)
actual_off_def.query('id == 41 or id == 152 or id == 201 or id == 326 or id == 2390')


,id,team_wk4,Off_wk4,Def_wk4
8,2390,Miami,592,365
26,41,UConn,542,250
36,152,NC State,436,523
53,201,Oklahoma,222,345


In [86]:
# import and join several CSV files into a joined DataFrame
csv_week1 = pd.read_csv('2024_cfb_JSON-CSV/2024-week1-teams-off-def.csv')
csv_week2 = pd.read_csv('2024_cfb_JSON-CSV/2024-week2-teams-off-def.csv')
csv_week3 = pd.read_csv('2024_cfb_JSON-CSV/2024-week3-teams-off-def.csv')
csv_week4 = pd.read_csv('2024_cfb_JSON-CSV/2024-week4-teams-off-def.csv')

week1_2 = csv_week1.merge(csv_week2, how='outer', on='id')
week1_3 = week1_2.merge(csv_week3, how='outer', on='id')
week1_3
week1_4 = week1_3.merge(csv_week4, how='outer', on='id')
week1_4


,id,team,Off_wk1,Def_wk1,team_wk2,Off_wk2,Def_wk2,team_wk3,Off_wk3,Def_wk3,team_wk4,Off_wk4,Def_wk4
0,2,Auburn,628.0,240.0,Auburn,286.0,334.0,Auburn,503.0,448.0,Auburn,431.0,334.0
1,5,UAB,517.0,177.0,UAB,259.0,296.0,UAB,354.0,427.0,NaN,NaN,NaN
2,6,South Alabama,582.0,550.0,South Alabama,335.0,404.0,South Alabama,622.0,237.0,South Alabama,474.0,385.0
3,8,Arkansas,687.0,130.0,Arkansas,648.0,385.0,Arkansas,427.0,354.0,Arkansas,334.0,431.0
4,9,Arizona State,499.0,118.0,Arizona State,428.0,298.0,Arizona State,347.0,400.0,Arizona State,376.0,337.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,2755,Grambling,241.0,481.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221,2771,Merrimack,217.0,237.0,Merrimack,279.0,624.0,NaN,NaN,NaN,NaN,NaN,NaN
222,2815,Lindenwood,202.0,530.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,2837,Texas A&M-Commerce,180.0,468.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
# find certain team(s)
week1_4.query('id == 41 or id == 152 or id == 201 or id == 326 or id == 2390')


,id,team,Off_wk1,Def_wk1,team_wk2,Off_wk2,Def_wk2,team_wk3,Off_wk3,Def_wk3,team_wk4,Off_wk4,Def_wk4
16,41,UConn,310.0,629.0,UConn,624.0,279.0,UConn,314.0,409.0,UConn,542.0,250.0
53,152,NC State,521.0,361.0,NC State,143.0,460.0,NC State,361.0,324.0,NC State,436.0,523.0
69,201,Oklahoma,378.0,197.0,Oklahoma,252.0,318.0,Oklahoma,349.0,279.0,Oklahoma,222.0,345.0
109,326,Texas State,486.0,288.0,Texas State,511.0,334.0,Texas State,400.0,347.0,NaN,NaN,NaN
162,2390,Miami,529.0,261.0,Miami,549.0,190.0,Miami,750.0,115.0,Miami,592.0,365.0


In [88]:
# Write all weeks DataFrame to a CSV file
week1_4.to_csv('2024_cfb_JSON-CSV/2024-weeks1-4-off-def.csv', index=False)

